In [0]:

# 1. LIMPIAR tablas
spark.sql("DROP TABLE IF EXISTS workspace.default.student_performance_bronze")
spark.sql("DROP TABLE IF EXISTS workspace.default.student_performance_silver")

print("🧹 Limpieza OK")


🧹 Limpieza OK


In [0]:
# 2. LEER CSV y RENOMBRAR columnas (solución al error)
ruta_csv = "/Volumes/workspace/default/student_performance_raw/StudentsPerformance.csv"

df_raw = spark.read.option("header", "true").option("inferSchema", "true").csv(ruta_csv)

# Renombrar columnas problemáticas
df_bronze = df_raw.select(
    df_raw["gender"].alias("gender"),
    df_raw["race/ethnicity"].alias("race_ethnicity"), 
    df_raw["parental level of education"].alias("parental_level_education"),
    df_raw["lunch"].alias("lunch"),
    df_raw["test preparation course"].alias("test_preparation_course"),
    df_raw["math score"].alias("math_score"),
    df_raw["reading score"].alias("reading_score"),
    df_raw["writing score"].alias("writing_score")
)

print(f"✅ Bronze listo: {df_bronze.count()} filas")
df_bronze.show(5)
df_bronze.printSchema()



✅ Bronze listo: 1000 filas
+------+--------------+------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race_ethnicity|parental_level_education|       lunch|test_preparation_course|math_score|reading_score|writing_score|
+------+--------------+------------------------+------------+-----------------------+----------+-------------+-------------+
|female|       group B|       bachelor's degree|    standard|                   none|        72|           72|           74|
|female|       group C|            some college|    standard|              completed|        69|           90|           88|
|female|       group B|         master's degree|    standard|                   none|        90|           95|           93|
|  male|       group A|      associate's degree|free/reduced|                   none|        47|           57|           44|
|  male|       group C|            some college|    standard|                   none|        76|  

In [0]:
# 3. GUARDAR BRONZE (ahora sin espacios = sin error)
df_bronze.write.mode("overwrite").saveAsTable("workspace.default.student_performance_bronze")

spark.sql("SELECT COUNT(*) FROM workspace.default.student_performance_bronze").show()



+--------+
|COUNT(*)|
+--------+
|    1000|
+--------+



In [0]:
# 4. CREAR SILVER (features)
from pyspark.sql import functions as F
from pyspark.sql.window import Window

df_silver = df_bronze.withColumn("promedio_final", 
                                (F.col("math_score") + F.col("reading_score") + F.col("writing_score")) / 3.0) \
                     .withColumn("estado_final", 
                                F.when(F.col("promedio_final") >= 70, "Aprobado").otherwise("Reprobado")) \
                     .withColumn("cuartil_rendimiento", 
                                F.ntile(4).over(Window.orderBy("promedio_final")))

df_silver.write.mode("overwrite").saveAsTable("workspace.default.student_performance_silver")

print("✅ SILVER creada!")
df_silver.show(5)


/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


✅ SILVER creada!
+------+--------------+------------------------+------------+-----------------------+----------+-------------+-------------+------------------+------------+-------------------+
|gender|race_ethnicity|parental_level_education|       lunch|test_preparation_course|math_score|reading_score|writing_score|    promedio_final|estado_final|cuartil_rendimiento|
+------+--------------+------------------------+------------+-----------------------+----------+-------------+-------------+------------------+------------+-------------------+
|female|       group C|        some high school|free/reduced|                   none|         0|           17|           10|               9.0|   Reprobado|                  1|
|female|       group B|             high school|free/reduced|                   none|         8|           24|           23|18.333333333333332|   Reprobado|                  1|
|  male|       group B|             high school|free/reduced|                   none|        30|  

In [0]:
%sql 
-- 5. VALIDAR TODO
SELECT 
  COUNT(*) total,
  ROUND(AVG(promedio_final), 1) prom_general,
  SUM(CASE WHEN estado_final = 'Aprobado' THEN 1 ELSE 0 END) aprobados
FROM workspace.default.student_performance_silver;


total,prom_general,aprobados
1000,67.8,459


In [0]:
%sql
-- Promedio por género (¡AHORA SÍ!)
SELECT gender, ROUND(AVG(math_score),1) prom_math, COUNT(*) n
FROM workspace.default.student_performance_silver
GROUP BY gender;


gender,prom_math,n
female,63.6,518
male,68.7,482


Databricks visualization. Run in Databricks to view.